In [1]:
import os
import sys
# sys.path=['../efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',]+sys.path
import pandas as pd
import numpy as np
from sklearn import metrics
from tqdm import tqdm
import torch
import torch.nn as nn
from efficientnet_pytorch import model as enet
import random
from sklearn.model_selection import StratifiedKFold
import codecs
from ignite.metrics import Recall, Precision, Accuracy

In [2]:
def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

random_state = set_seed(2021)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [4]:
class ClassificationDataset:
    
    def __init__(self, image_paths, targets): 
        self.image_paths = image_paths
        self.targets = targets

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item]).astype(float)

        targets = self.targets[item]
                
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

In [5]:
df_train=pd.read_csv('../data/train/train_labels.csv')
df_train.head()

,id,target
0,0000799a2b2c42d,0
1,00042890562ff68,0
2,0005364cdcb8e5b,0
3,0007a5a46901c56,0
4,0009283e145448e,0


In [6]:
df_train['img_path']=df_train['id'].apply(lambda x:f'../data/train/{x[0]}/{x}.npy')
df_train.head()

,id,target,img_path
0,0000799a2b2c42d,0,../data/train/0/0000799a2b2c42d.npy
1,00042890562ff68,0,../data/train/0/00042890562ff68.npy
2,0005364cdcb8e5b,0,../data/train/0/0005364cdcb8e5b.npy
3,0007a5a46901c56,0,../data/train/0/0007a5a46901c56.npy
4,0009283e145448e,0,../data/train/0/0009283e145448e.npy


In [7]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(6, 3, kernel_size=3, stride=1, padding=3, bias=False)

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [8]:
def train(data_loader, model, optimizer, scheduler, device):
    
    model.train()
    running_loss = 0.0
    i = 0

    accuracy = Accuracy()
    precision = Precision()
    recall = Recall()
    epoch_loss = 0
    
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        i = i + 1
        inputs = data["image"]
        targets = data['targets']
        
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()
        scheduler.step()

        predicted = torch.sigmoid(outputs).round()
        accuracy.update((predicted, targets))
        precision.update((predicted, targets))
        recall.update((predicted, targets))
        
        running_loss += loss.item()
        epoch_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (1 + 1, i + 1, running_loss / 100)) # 2000))
            running_loss = 0.0

    avg_epoch_loss = epoch_loss / len(data_loader)
    print(f"""Training Error: \n 
        Accuracy: {100*accuracy.compute():>2f}%, \n
        Recall: {100*recall.compute():>2f}%, \n
        Precision: {100*precision.compute():>2f}%, \n
        Avg loss: {avg_epoch_loss:>2f} \n
    """)
        
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data["image"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

In [9]:
baseline_name = 'efficientnet-b1'
pretrained_model = {
    baseline_name: '../efficientnet-pytorch/efficientnet-b1-dbc7070a.pth'
}
models = []
device = "cuda"
epochs = 2 # 20
Batch_Size = 8 # 32
learn_rate = 1e-3
min_learn_rate = 1e-6

X = df_train.img_path.values
Y = df_train.target.values
skf = StratifiedKFold(n_splits=5)
fold = 0

for train_index, test_index in skf.split(X, Y):
    
    model = enetv2(baseline_name, out_dim=1)
    model.to(device)

    train_images, valid_images = X[train_index], X[test_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]

    train_dataset = ClassificationDataset(image_paths=train_images, targets=train_targets)
    valid_dataset = ClassificationDataset(image_paths=valid_images, targets=valid_targets)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size,shuffle=True) #, num_workers=4)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size,shuffle=False) #, num_workers=4)

    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs, eta_min=min_learn_rate, last_epoch=-1
    )
    

    best_roc_auc = 0
    for epoch in range(epochs):
        train(train_loader, model, optimizer, scheduler, device=device)
        predictions, valid_targets = evaluate(valid_loader, model, device=device)
        roc_auc = metrics.roc_auc_score(valid_targets, predictions)
        print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")
        
        with codecs.open('log.log', 'a') as up:
            up.write(f"Fold={fold}, Epoch={epoch}, Valid ROC AUC={roc_auc}/{best_roc_auc}\n")

        if roc_auc > best_roc_auc:        
            torch.save(model.state_dict(), baseline_name + '-' + str(fold) + '-' + str(epoch) + '.pt')
            best_roc_auc = roc_auc
            
    torch.save(model.state_dict(),baseline_name + '-' + str(fold) + '.pt')
    models.append(model)
    fold += 1
    break

Training:   2%|▏         | 99/6000 [00:42<37:01,  2.66it/s] 

[2,   100] loss: 0.411


Training:   3%|▎         | 199/6000 [01:20<40:16,  2.40it/s]

[2,   200] loss: 0.409


Training:   5%|▍         | 299/6000 [01:59<35:30,  2.68it/s]

...

[2,  6000] loss: 0.318


Training: 100%|██████████| 6000/6000 [46:12<00:00,  2.16it/s]


Training Error: 
 
        Accuracy: 91.372917%, 

        Recall: 19.145833%, 

        Precision: 77.947413%, 

        Avg loss: 0.281964 

    


Evaluating: 100%|██████████| 1500/1500 [04:15<00:00,  5.88it/s]


Epoch=1, Valid ROC AUC=0.6065740740740743
